### Q1: Getting the data

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
# get the first index of the documents above
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

### Q2: Indexing the data

##### Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

In [3]:
#import Elastic search class
from elasticsearch import Elasticsearch

In [4]:
#estimator object of elastic search with port. ensure elastic search container is running
es_client = Elasticsearch('http://localhost:9200')

In [5]:
#call elastic search method
es_client.info()

ObjectApiResponse({'name': '3f4bf6a5221a', 'cluster_name': 'docker-cluster', 'cluster_uuid': '3QpKe2FUSXWfFBxwj4YqHQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
# create an index for elastic search engine
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "llm-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'llm-questions'})

In [7]:
# to track progress bar
from tqdm.auto import tqdm

In [8]:
# insert all documents into elastic search
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

### Q3: Searching

##### Now let's search in our index.

##### We will execute a query "How do I execute a command in a running docker container?".

##### Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

##### What's the score for the top ranking result?

In [9]:
query = "How do I execute a command in a running docker container?"

In [10]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [11]:
#search query in the Elastic search engine
response = es_client.search(index=index_name, body=search_query)

In [12]:
response['hits']['hits']

[{'_index': 'llm-questions',
  '_id': 'ZoMqQ5ABxBO6zzbdV18m',
  '_score': 75.54128,
  '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n|--------+------------------+-------+-------|\n| public | yellow_taxi_data | table | root  |\n+--------+------------------+-------+-------+\nSELECT 1\nTime: 0.009s\nroot@pg-database:ny_taxi>',
   'sec

In [13]:
result_doc = []

for hit in response['hits']['hits']:
    result_doc.append(hit)

In [14]:
# score of top ranking result
result_doc[0]['_score']

75.54128

### Q4: Filtering

##### Now let's only limit the questions to machine-learning-zoomcamp.

##### Return 3 results. What's the 3rd question returned by the search engine

In [15]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [16]:
#search query in the Elastic search engine
response = es_client.search(index=index_name, body=search_query)

In [17]:
response['hits']['hits']

[{'_index': 'llm-questions',
  '_id': 'WIMqQ5ABxBO6zzbdaGHj',
  '_score': 84.050095,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'llm-questions',
  '_id': 'd4MqQ5ABxBO6zzbdaWHQ',
  '_score': 51.04628,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_dir

In [18]:
result_doc = []

for hit in response['hits']['hits']:
    result_doc.append(hit)

In [19]:
# What's the 3rd question returned by the search engine
result_doc[2]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

### Q5: Building a prompt

##### Now we're ready to build a prompt to send to an LLM.

##### Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

##### What's the length of the resulting prompt? (use the len function)

In [20]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [21]:
question = 'How do I copy files from a different folder into docker container’s working directory?'

In [22]:
# putting the above in a function:

def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": question,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [23]:
search_results = elastic_search(question)

In [24]:
search_results

[{'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'questi

In [25]:
# prompt function
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"question: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=question, context=context).strip()
    return prompt

In [26]:
# combining all functions 
search_results = elastic_search(question)
prompt = build_prompt(question, search_results)

In [27]:
len(prompt)

1528

### Q6: Tokens

##### Let's calculate the number of tokens in our query with tiktoken:

##### Use the encode function. How many tokens does our prompt have?

In [28]:
import tiktoken

In [29]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [31]:
len(encoding.encode(text=prompt))

328